In [1]:
import gc
import numpy as np
"""
ToyNetwork 재분석
    램프 : X
    집계시간 : 5분
    분석시간 1800~8400
    검지기 : 840개
"""
import pandas as pd

import os

# FIX 값 모음
###################################################################################################################

path = r"C:\VISSIM_Workspace\테스트\램프X\mer 통합"

start_interval = 1800
end_interval = 8400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]

###################################################################################################################

# 함수 모음
###################################################################################################################

# 속도 변화율
def speed_mean(original_df):
    copy_df = original_df.copy()

    # 램프 검지기 제외
    copy_df = copy_df[~copy_df["New_Measurement"].between(900, 910)]

    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도 변화율
def density_mean(speed_df):
    copy_df = speed_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 중차량 혼입률
def heavy_rate(original_df):
    copy_df = original_df.copy()

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup"])
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup"])
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup"],
        how="left"
    )

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 동적 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()

    # 실측용량 C(3차로 6600)
    max_capacity = 6600
    entry_saturation_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df

# 램프 간섭 영향률
def rfr_rate(original_df):
    rfr_df = (
        original_df[original_df["New_Measurement"] == 1] # 60
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="main_line")
    )
    input_df = (
        original_df[original_df["New_Measurement"] == 2] # 59
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="input_ramp")
    )
    output_df = (
        original_df[original_df["New_Measurement"] == 3] # 61
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="output_ramp")
    )
    #main_df["RFR"] = (input_df["input_ramp"] + output_df["output_ramp"])/main_df["main_line"]
    # 유출램프 구현이 안 됐기 때문에 0값으로 fix
    rfr_df = rfr_df.drop(columns=["New_Measurement", "main_line"])
    rfr_df["RFR"] = 0
    return rfr_df

# 진출 원활율
def output_normality(original_df):

    copy_df = original_df.copy()
    #copy_df["F(outrate)"] = 0.88
    copy_df["F(outrate)"] = 0
    return copy_df


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged_df = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "rate"]], on=["TimeGroup"])
    .merge(entry_saturation_df[["TimeGroup", "New_Measurement", "Phi_진입"]], on=["TimeGroup", "New_Measurement"])
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "F(outrate)"]], on="TimeGroup")
    )

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["F(outrate)"]
    )
    merged_df = modify_frame(merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    modify_df = df.copy()

    modify_df["StartTime"] = modify_df["TimeGroup"].str.split("~").str[0].astype(int)
    modify_df = modify_df[(modify_df["StartTime"] >=1800) &(modify_df["StartTime"] < 5400)]
    modify_df = modify_df[modify_df["New_Measurement"] != 280]
    return modify_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = path
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 속도변화율
                speed_df = speed_mean(original_df)

                # 밀도변화율
                density_df = density_mean(speed_df)

                # 중차량 혼입률
                heavy_df = heavy_rate(original_df)
                #display("heavy_df : ", heavy_df)

                # 동적 포화도
                entry_saturation_df = entry_saturation(original_df)
                #display("entry_saturation_df : ", entry_saturation_df)

                # 램프 간섭 영향률
                rfr_df = rfr_rate(original_df)

                # 진출 원활율
                normality_df = output_normality(rfr_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                #display("stvm_df : ", stvm_df)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                display("z_score_df : ", z_score_df)
                save_to_excel(z_score_df, folder_path, "환산점수", i)

                # STVM 피봇
                #stvm_pivot_df = pivot_table(stvm_df, "STVM")
                #display("stvm_pivot_df : ", stvm_pivot_df)
                #save_to_excel(stvm_pivot_df, folder_path, "STVM", i)

                # 속도값 피봇
                #speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #display("speed_pivot_df : ", speed_pivot_df)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 메모리 정리
                #del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,91.288129,90.495809,94.598995,89.631776,85.917441,88.846527,88.846799,90.557601,83.885123,72.065802,...,57.537307,54.636554,52.513820,57.897891,53.281567,54.388796,67.388271,64.281633,55.886360,56.939333
2100~2400,72.388941,83.219058,83.220140,80.896933,88.573981,87.338417,88.221407,87.198417,85.152604,94.309862,...,71.645516,81.518792,81.279395,82.359860,85.627931,85.431617,71.804453,71.546105,71.591103,82.603986
2400~2700,71.407049,68.771557,70.680155,63.939837,67.966062,65.562731,64.351658,70.864702,71.045553,61.978021,...,67.235750,70.852990,71.039154,70.320655,72.233187,69.662657,82.513838,71.109697,72.079574,70.551907
2700~3000,81.913406,72.091305,85.591322,92.667044,72.323788,72.382463,88.518839,80.647096,80.802114,86.542424,...,96.269765,95.862432,90.489721,95.895912,93.857988,97.625710,96.243764,97.049739,95.028884,97.491240
3000~3300,52.527076,62.615541,54.733379,55.444854,60.360672,55.798914,55.201354,52.809893,58.874513,52.812680,...,63.041931,56.660558,70.209445,56.467301,60.107481,56.822109,53.510194,52.550999,59.944476,54.810300
3300~3600,71.228417,61.653275,60.002195,61.297931,60.419646,63.556004,64.415358,70.777868,60.899671,64.153537,...,90.542707,95.828296,95.398133,94.993224,92.622573,92.613794,93.218445,95.414717,91.894430,93.275617
3600~3900,70.110539,70.110145,71.375398,71.308167,71.582152,82.326770,72.091274,72.091962,71.480727,84.376871,...,68.875552,68.529366,69.609209,71.445559,72.271021,67.600412,72.279515,72.483801,71.192628,81.053029
3900~4200,72.207089,81.241189,70.047142,70.736940,71.076788,71.074697,62.552946,58.322940,70.843227,70.651062,...,53.090762,52.783933,52.717411,52.703637,52.814569,53.780191,52.751573,52.891850,54.353842,52.591860
4200~4500,72.193609,72.192063,83.602541,71.515297,81.239475,71.227782,88.991074,92.652752,81.657963,85.213201,...,72.328346,81.492953,81.229566,82.307538,85.571407,72.094415,80.997662,89.598928,82.984772,84.317133


환산점수_1.xlsx 생성 완료


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,85.977211,86.986927,82.646795,85.800596,86.657633,84.705139,82.254665,71.738869,72.367009,71.122285,...,71.391449,70.624512,70.666983,71.266661,71.088349,71.088338,64.405687,60.847325,65.158393,64.467242
2100~2400,55.448371,56.055335,59.412518,52.751205,56.070488,53.200415,57.477074,57.689760,60.196978,63.935690,...,71.299515,71.034149,72.472091,71.123475,71.029331,72.049375,85.002245,83.442505,71.172949,70.326316
2400~2700,66.424114,66.697116,69.019825,71.298264,67.342476,72.415190,71.776246,69.741290,70.162634,63.303379,...,96.401155,96.032067,92.473696,93.016883,94.138653,92.114677,91.831547,93.912683,96.651885,97.338742
2700~3000,82.666771,80.672723,72.157466,72.474909,72.058104,80.511044,81.506198,85.497070,71.441629,85.185157,...,94.673774,95.837625,96.531330,93.461573,88.747195,84.881772,89.848693,83.523182,71.962416,71.948758
3000~3300,71.466006,71.711842,71.545290,70.279186,72.492084,71.801187,72.222217,72.476959,72.313047,71.063706,...,61.409225,60.263973,62.953180,70.797542,81.724631,85.828864,71.004180,86.724078,88.785967,89.476929
3300~3600,81.263172,83.189283,72.164672,86.207929,71.845019,72.070191,72.235851,83.811959,86.462257,90.183026,...,63.363250,67.501796,65.212839,60.091956,59.677915,62.156348,64.309812,59.187308,62.164810,63.016946
3600~3900,54.772400,56.306182,58.426006,55.456958,59.621546,59.206742,57.493159,54.381315,54.869231,53.860403,...,59.367199,57.103582,55.055904,54.581487,52.649597,52.855450,52.738612,52.606511,52.604147,52.676877
3900~4200,89.035721,83.269452,90.260896,85.667865,85.316409,72.426238,72.217707,81.800850,71.542006,81.451643,...,95.342740,95.896745,95.892804,97.517260,97.700533,97.955071,97.823424,96.773921,91.844822,90.651087
4200~4500,86.234038,88.394000,84.655583,85.717858,84.265334,88.885388,86.066875,89.178452,94.392359,89.277920,...,71.207545,68.205572,63.950741,67.518204,70.915159,70.629401,68.472791,70.375400,72.342917,82.577843


...

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,82.123674,82.992892,82.170857,84.623800,83.408653,85.789731,84.049003,85.211155,84.635631,85.781043,...,53.665564,54.703096,56.295534,57.278869,58.056379,57.135672,58.096984,60.990807,59.212217,56.658035
2100~2400,59.246415,60.146448,62.826079,61.021650,61.655068,59.987142,60.638705,59.571670,60.660765,62.773833,...,97.829646,97.260320,96.473867,95.764666,94.092390,93.969195,93.634443,92.225409,91.728526,91.526359
2400~2700,90.514823,90.520007,88.511136,86.832832,85.094270,85.068116,85.624438,85.634315,84.471325,81.350642,...,71.038016,71.185471,71.006952,70.484224,70.503076,70.835828,71.510297,81.395027,81.322106,84.661548
2700~3000,54.133321,53.592845,53.329604,53.874805,54.342360,55.107749,54.232630,53.782356,54.083132,54.580593,...,59.384910,59.223789,58.755012,58.755664,58.290943,57.035834,55.223450,53.199153,53.353525,52.932219
3000~3300,95.403870,95.202205,95.804118,95.390366,95.781919,96.250604,96.719260,96.974809,96.328843,95.401772,...,80.048098,72.153683,72.247643,72.243015,72.411076,80.934707,81.581224,82.235389,84.080086,82.554937
3300~3600,87.950564,91.281414,91.432672,92.202266,91.823669,90.135805,89.570676,89.549522,89.053150,89.036358,...,58.526791,58.520435,57.953961,57.994868,58.929514,60.268432,62.887900,64.371037,65.741254,69.839675
3600~3900,59.691912,57.814366,58.330980,57.839833,59.265541,59.172351,58.254619,57.836332,59.741881,62.263497,...,85.193814,85.684357,86.231987,87.317516,87.346692,87.346678,85.126575,85.068385,86.755057,85.359063
3900~4200,93.128917,84.789284,83.887173,84.173878,81.683754,80.998577,82.891065,86.446361,85.923836,82.966846,...,55.178608,55.599520,55.968140,55.130953,54.596349,53.364179,52.687678,52.542919,52.579189,52.548950
4200~4500,86.213229,84.542237,71.639301,70.448617,69.639924,68.807777,68.108638,65.533376,65.630559,66.260790,...,98.433683,98.476490,98.476078,98.479401,98.245137,98.222036,98.221673,98.298972,98.306436,98.350275


환산점수_4.xlsx 생성 완료


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,270,271,272,273,274,275,276,277,278,279
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,70.225061,57.930120,69.169187,64.424656,71.922197,71.323525,70.433326,81.334579,71.082246,71.083136,...,58.858789,91.584594,95.155647,72.379227,65.830683,71.408183,71.406034,71.406107,71.407191,88.631090
2100~2400,56.721376,70.775399,55.389815,59.965735,59.964976,59.966378,69.022185,53.427977,52.686423,56.091685,...,91.326753,82.355987,85.317600,89.022582,95.172933,92.480489,87.053083,92.476994,83.135928,80.695842
2400~2700,71.995595,72.310691,84.894500,69.246827,62.584525,70.489285,71.679616,71.992521,91.231595,91.286178,...,57.165788,57.903198,53.609601,53.132324,54.560799,52.665271,52.818771,52.738691,54.698289,53.203597
2700~3000,70.150509,69.286694,68.000949,71.547824,70.673123,68.004367,66.759099,71.230462,66.755646,65.538838,...,70.165449,72.178273,70.181977,83.269821,67.340442,64.643070,80.940308,71.006624,70.997383,87.361312
3000~3300,93.521152,96.443923,97.695732,96.358897,97.905605,95.860096,93.483068,94.210507,95.449473,95.447693,...,53.073838,53.198394,52.787066,52.750804,52.791742,58.490262,52.607180,52.942815,53.225489,52.561722
3300~3600,57.737242,61.414688,52.704868,58.747778,53.999597,62.716849,64.588844,62.382609,59.424526,59.428536,...,61.319816,85.623293,71.668265,64.539417,70.734240,53.200883,63.304893,70.747077,70.106886,66.875543
3600~3900,97.777847,90.736992,97.886806,91.622423,92.662137,94.253280,94.252794,81.146285,91.310815,84.304321,...,96.487308,90.757532,88.800491,92.898732,72.096888,94.963893,95.820666,90.119770,85.484147,87.093889
3900~4200,92.687577,89.777055,85.047569,90.202000,93.045291,83.806918,85.510387,97.543388,80.874125,92.357041,...,93.150220,96.360085,96.802856,97.802250,96.634232,97.652876,88.044037,95.267340,90.402644,71.587421
4200~4500,55.651922,61.549799,67.213473,64.452669,65.777970,62.412113,53.696297,59.510229,68.394238,65.807432,...,69.998548,72.446126,84.226778,71.611038,91.546835,72.268980,86.437704,83.507174,87.521437,91.677736


환산점수_5.xlsx 생성 완료
